<a href="https://colab.research.google.com/github/DataLama/triton-tutorials/blob/main/tutorials/basic/1_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip show torch
!pip show triton

Name: torch
Version: 2.2.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision
Name: triton
Version: 2.2.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock
Required-by: torch


----

In [15]:
%%writefile 1_grid.py
from typing import Dict, Tuple, Optional
from argparse import ArgumentParser

import triton
import triton.language as tl

@triton.jit
def print_grid():
  x_pid = tl.program_id(0) # x축의 프로세스 아이디
  y_pid = tl.program_id(1) # y축의 프로세스 아이디
  z_pid = tl.program_id(2) # z축의 프로세스 아이디
  tl.device_print("x_pid: ", x_pid)
  tl.device_print("y_pid: ", y_pid)
  tl.device_print("z_pid: ", z_pid)

if __name__ == "__main__":
  parser = ArgumentParser()
  parser.add_argument("--x_grid", type=int, default=1)
  parser.add_argument("--y_grid", type=int, default=1)
  parser.add_argument("--z_grid", type=int, default=1)
  args = parser.parse_args()

  x_grid = args.x_grid
  y_grid = args.y_grid
  z_grid = args.z_grid


  def grid(meta:Dict)->Tuple[int, Optional[int], Optional[int]]:
      """
      Args: meta는 그리드를 결정하는데 사용할 수 있는 meta 정보
      """
      return (x_grid, y_grid, z_grid)

  print_grid[grid]()

Overwriting 1_grid.py


In [18]:
# grid (1,)
!python 1_grid.py --x_grid=1 | wc -l

384


In [19]:
# grid (4, 2)
!python 1_grid.py --x_grid=4 --y_grid=2 | wc -l

3072


In [20]:
!python 1_grid.py --x_grid=4 --y_grid=2 | sort | uniq

pid (0, 0, 0) idx () x_pid: 0
pid (0, 0, 0) idx () y_pid: 0
pid (0, 0, 0) idx () z_pid: 0
pid (0, 1, 0) idx () x_pid: 0
pid (0, 1, 0) idx () y_pid: 1
pid (0, 1, 0) idx () z_pid: 0
pid (1, 0, 0) idx () x_pid: 1
pid (1, 0, 0) idx () y_pid: 0
pid (1, 0, 0) idx () z_pid: 0
pid (1, 1, 0) idx () x_pid: 1
pid (1, 1, 0) idx () y_pid: 1
pid (1, 1, 0) idx () z_pid: 0
pid (2, 0, 0) idx () x_pid: 2
pid (2, 0, 0) idx () y_pid: 0
pid (2, 0, 0) idx () z_pid: 0
pid (2, 1, 0) idx () x_pid: 2
pid (2, 1, 0) idx () y_pid: 1
pid (2, 1, 0) idx () z_pid: 0
pid (3, 0, 0) idx () x_pid: 3
pid (3, 0, 0) idx () y_pid: 0
pid (3, 0, 0) idx () z_pid: 0
pid (3, 1, 0) idx () x_pid: 3
pid (3, 1, 0) idx () y_pid: 1
pid (3, 1, 0) idx () z_pid: 0
